In [1]:
import os
import sys
HOME_DIR = '/Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering'
os.chdir(HOME_DIR)

DATASETS_DIR = f'{HOME_DIR}/datasets'
DATA_DIR = f'{DATASETS_DIR}/11'

TRAIN_CSV_FILE = f'{DATA_DIR}/bounding_boxes/train_labels.csv'
TRAIN_IMAGES_DIR = f'{DATA_DIR}/images/train'
TRAIN_LABELS_DIR = f'{DATA_DIR}/labels/train'
SELECTED_TRAIN_IMAGES_DIR= f'{DATA_DIR}/images/train_selected'
SELECTED_TRAIN_LABELS_DIR = f'{DATA_DIR}/labels/train_selected'


TEST_CSV_FILE = f'{DATA_DIR}/bounding_boxes/test_labels.csv'
TEST_IMAGES_DIR = f'{DATA_DIR}/images/test'
TEST_LABELS_DIR = f'{DATA_DIR}/labels/test'
SELECTED_TEST_IMAGES_DIR = f'{DATA_DIR}/images/test_selected'
SELECTED_TEST_LABELS_DIR = f'{DATA_DIR}/labels/test_selected'

OUTPUT_IMAGES_DIR = 'week-06-portfolio/evaluation_images'

yaml_file_path = f'{DATA_DIR}/graffiti.yaml'

print(f'HOME_DIR: {HOME_DIR}')
print(f'DATASETS_DIR: {DATASETS_DIR}')
print(f'DATA_DIR: {DATA_DIR}')
print(f'TRAIN_CSV_FILE: {TRAIN_CSV_FILE}')
print(f'TRAIN_IMAGES_DIR: {TRAIN_IMAGES_DIR}')
print(f'TRAIN_LABELS_DIR: {TRAIN_LABELS_DIR}')
print(f'TEST_CSV_FILE: {TEST_CSV_FILE}')
print(f'TEST_IMAGES_DIR: {TEST_IMAGES_DIR}')
print(f'TEST_LABELS_DIR: {TEST_LABELS_DIR}')

HOME_DIR: /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering
DATASETS_DIR: /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets
DATA_DIR: /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11
TRAIN_CSV_FILE: /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/bounding_boxes/train_labels.csv
TRAIN_IMAGES_DIR: /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/images/train
TRAIN_LABELS_DIR: /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/labels/train
TEST_CSV_FILE: /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/bounding_boxes/test_labels.csv
TEST_IMAGES_DIR: /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/images/test
TEST_LABELS_DIR: /Users/anhdang/Documents/Github/COS

In [11]:
!pip install --upgrade pip --quiet
!pip install -r requirements.txt --upgrade --quiet

In [2]:
from ultralytics import YOLO
import platform
import torch
import pandas as pd
import os
from tqdm import tqdm
import torch
from torchvision.ops import box_iou
import pandas as pd
from PIL import Image
from tqdm import tqdm
import random
import shutil
import cv2
import torch
from torchvision.ops import box_iou
import pandas as pd
from PIL import Image
from tqdm import tqdm
import requests
import re
import subprocess
import json


In [3]:
if platform.system() == 'Darwin':
    device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: mps


In [4]:
def convert_annotations(csv_file, images_dir, output_dir, class_mapping):
    df = pd.read_csv(csv_file)
    grouped = df.groupby('filename')

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename, group in tqdm(grouped, desc=f'Converting annotations for {csv_file}'):
        image_path = os.path.join(images_dir, filename)
        if not os.path.exists(image_path):
            continue  # Skip if image does not exist

        img_width = group.iloc[0]['width']
        img_height = group.iloc[0]['height']

        annotations = []
        for _, row in group.iterrows():
            class_id = class_mapping[row['class']]
            xmin = row['xmin']
            ymin = row['ymin']
            xmax = row['xmax']
            ymax = row['ymax']

            # Convert to YOLO format
            x_center = ((xmin + xmax) / 2) / img_width
            y_center = ((ymin + ymax) / 2) / img_height
            bbox_width = (xmax - xmin) / img_width
            bbox_height = (ymax - ymin) / img_height

            annotations.append(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}")

        # Write annotations to file
        txt_filename = os.path.splitext(filename)[0] + '.txt'
        with open(os.path.join(output_dir, txt_filename), 'w') as f:
            for ann in annotations:
                f.write(ann + '\n')



In [5]:
# Define class mapping
class_mapping = {'Graffiti': 0}

# Convert training annotations
convert_annotations(TRAIN_CSV_FILE, TRAIN_IMAGES_DIR, TRAIN_LABELS_DIR, class_mapping)

# Convert test annotations
convert_annotations(TEST_CSV_FILE, TEST_IMAGES_DIR, TEST_LABELS_DIR, class_mapping)


Converting annotations for /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/bounding_boxes/train_labels.csv: 100%|██████████| 813/813 [00:00<00:00, 5149.12it/s]
Converting annotations for /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/bounding_boxes/test_labels.csv: 100%|██████████| 209/209 [00:00<00:00, 5497.54it/s]


In [6]:
def select_random_images(source_dir, dest_dir, num_images, used_images=set()):
    images = [f for f in os.listdir(source_dir) if f.endswith('.jpg') and f not in used_images]
    selected_images = random.sample(images, min(num_images, len(images)))
    used_images.update(selected_images)

    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    
    for file in os.listdir(dest_dir):
        file_path = os.path.join(dest_dir, file)
        if os.path.isfile(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    
    for img in selected_images:
        shutil.copy(os.path.join(source_dir, img), os.path.join(dest_dir, img))
    return used_images

def copy_annotation_files(image_dir, label_source_dir, label_dest_dir):
    if not os.path.exists(label_dest_dir):
        os.makedirs(label_dest_dir)
        
    for file in os.listdir(label_dest_dir):
        file_path = os.path.join(label_dest_dir, file)
        if os.path.isfile(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    
    
    for img_file in os.listdir(image_dir):
        if img_file.endswith('.jpg'):
            base_name = os.path.splitext(img_file)[0]
            label_file = base_name + '.txt'
            src_label_path = os.path.join(label_source_dir, label_file)
            dst_label_path = os.path.join(label_dest_dir, label_file)
            if os.path.exists(src_label_path):
                shutil.copy(src_label_path, dst_label_path)

In [7]:
# Set random seed for reproducibility
random.seed(42)

# Initialize used images sets
used_train_images = set()
used_test_images = set()

# Select 400 random training images
used_train_images = select_random_images(TRAIN_IMAGES_DIR, SELECTED_TRAIN_IMAGES_DIR, 400, used_train_images)

# Copy corresponding training annotation files
copy_annotation_files(SELECTED_TRAIN_IMAGES_DIR, TRAIN_LABELS_DIR, SELECTED_TRAIN_LABELS_DIR)

# Select 40 random test images
used_test_images = select_random_images(TEST_IMAGES_DIR, SELECTED_TEST_IMAGES_DIR, 40, used_test_images)

# Copy corresponding test annotation files
copy_annotation_files(SELECTED_TEST_IMAGES_DIR, TEST_LABELS_DIR, SELECTED_TEST_LABELS_DIR)

In [8]:
def create_yaml_file(file_path, train_images, val_images, nc, class_names):
    data = {
        'train': train_images,
        'val': val_images,
        'nc': nc,
        'names': class_names
    }
    with open(file_path, 'w') as f:
        for key, value in data.items():
            if isinstance(value, list):
                value_str = str(value).replace("'", "")
                f.write(f"{key}: {value_str}\n")
            else:
                f.write(f"{key}: {value}\n")

In [9]:
# Usage
train_images_path = os.path.abspath(SELECTED_TRAIN_IMAGES_DIR)
val_images_path = os.path.abspath(SELECTED_TEST_IMAGES_DIR)  # Using test data as validation set

nc = 1
class_names = ['Graffiti']

create_yaml_file(yaml_file_path, train_images_path, val_images_path, nc, class_names)

# Test Run

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv5s model
model = YOLO('week-06-portfolio/models/yolov5su.pt')

# Train the model
results = model.train(data=yaml_file_path, epochs=1, imgsz=640, batch=16, name='graffiti_detection', device=device)


In [12]:
def compute_iou(pred_boxes, true_boxes):
    # Convert boxes to tensors
    pred_boxes = torch.tensor(pred_boxes)
    true_boxes = torch.tensor(true_boxes)

    # Compute IoU
    iou = box_iou(pred_boxes, true_boxes)
    return iou.diag().numpy()  # Get IoUs for matched boxes

def evaluate_model(model, images_dir, labels_dir, output_images_dir=None):
    results = []
    images = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]

    if output_images_dir and not os.path.exists(output_images_dir):
        os.makedirs(output_images_dir)

    for img_name in tqdm(images, desc='Evaluating model'):
        img_path = os.path.join(images_dir, img_name)
        label_path = os.path.join(labels_dir, os.path.splitext(img_name)[0] + '.txt')

        # Perform inference
        preds = model.predict(img_path, conf=0.25)

        # Load image for drawing
        img = cv2.imread(img_path)
        img_height, img_width = img.shape[:2]

        # Get predicted boxes and confidence scores
        pred_boxes = []
        confidences = []
        for pred in preds:
            for box in pred.boxes:
                x_min = box.xyxy[0][0].item()
                y_min = box.xyxy[0][1].item()
                x_max = box.xyxy[0][2].item()
                y_max = box.xyxy[0][3].item()
                conf = box.conf.item()
                pred_boxes.append([x_min, y_min, x_max, y_max])
                confidences.append(conf)

                # Draw predicted bounding box
                if output_images_dir:
                    label = f"{model.names[int(box.cls)]}: {conf:.2f}"
                    cv2.rectangle(img, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 255, 0), 2)
                    cv2.putText(img, label, (int(x_min), int(y_min) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Get true boxes
        true_boxes = []
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    class_id, x_center, y_center, width, height = map(float, line.strip().split())
                    # Convert back to absolute coordinates
                    x_center *= img_width
                    y_center *= img_height
                    width *= img_width
                    height *= img_height
                    x_min = x_center - width / 2
                    y_min = y_center - height / 2
                    x_max = x_center + width / 2
                    y_max = y_center + height / 2
                    true_boxes.append([x_min, y_min, x_max, y_max])

                    # Draw true bounding box
                    if output_images_dir:
                        cv2.rectangle(img, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 0, 255), 2)

        # Save image with drawn bounding boxes
        if output_images_dir:
            output_image_path = os.path.join(output_images_dir, img_name)
            cv2.imwrite(output_image_path, img)

        # Compute IoU
        if pred_boxes and true_boxes:
            ious = compute_iou(pred_boxes, true_boxes)
            max_iou = max(ious)
            max_conf = confidences[ious.argmax()]
        else:
            max_iou = 0.0
            max_conf = 0.0 if not confidences else max(confidences)

        results.append({
            'image_name': img_name,
            'confidence_value': max_conf,
            'IoU_value': max_iou
        })

    return pd.DataFrame(results)



In [13]:
# Usage
df_results = evaluate_model(model, SELECTED_TEST_IMAGES_DIR, SELECTED_TEST_LABELS_DIR, OUTPUT_IMAGES_DIR)
df_results.to_csv('week-06-portfolio/evaluation_results.csv', index=False)

Evaluating model:   0%|          | 0/40 [00:00<?, ?it/s]


image 1/1 /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/images/test_selected/IMG_20180713_105001.jpg: 640x480 6 Graffitis, 1231.1ms
Speed: 19.1ms preprocess, 1231.1ms inference, 277.1ms postprocess per image at shape (1, 3, 640, 480)


Evaluating model:   2%|▎         | 1/40 [00:02<01:22,  2.12s/it]


image 1/1 /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/images/test_selected/IMG_20100101_032102.jpg: 640x480 44 Graffitis, 25.6ms
Speed: 2.1ms preprocess, 25.6ms inference, 241.7ms postprocess per image at shape (1, 3, 640, 480)


Evaluating model:   5%|▌         | 2/40 [00:02<00:41,  1.09s/it]

...

Evaluating model:  92%|█████████▎| 37/40 [00:10<00:00,  3.73it/s]


image 1/1 /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/images/test_selected/IMG_20180720_142111.jpg: 640x480 17 Graffitis, 25.8ms
Speed: 2.0ms preprocess, 25.8ms inference, 7.7ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/images/test_selected/IMG_20180724_121144.jpg: 640x480 9 Graffitis, 19.0ms
Speed: 1.9ms preprocess, 19.0ms inference, 17.2ms postprocess per image at shape (1, 3, 640, 480)


Evaluating model:  98%|█████████▊| 39/40 [00:10<00:00,  5.58it/s]


image 1/1 /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/images/test_selected/IMG_20180714_102857.jpg: 640x480 12 Graffitis, 22.3ms
Speed: 1.9ms preprocess, 22.3ms inference, 17.1ms postprocess per image at shape (1, 3, 640, 480)


Evaluating model: 100%|██████████| 40/40 [00:10<00:00,  3.77it/s]


# Full Train Base on the Requirements

In [15]:
# Initialize variables for iterative training
iou_threshold = 0.8
satisfied = False
iteration = 1

while not satisfied:
    print(f"\nStarting iteration {iteration}")

    # Select new training images
    training_images = random.sample(os.listdir(TRAIN_IMAGES_DIR), 400)

    # Update training data directories
    selected_train_images_dir = f'week-06-portfolio/images/train_selected_iter_{iteration}'
    selected_train_labels_dir = f'week-06-portfolio/labels/train_selected_iter_{iteration}'
    if not os.path.exists(selected_train_images_dir):
        os.makedirs(selected_train_images_dir)
    if not os.path.exists(selected_train_labels_dir):
        os.makedirs(selected_train_labels_dir)

    # Copy selected images and annotations
    for img in training_images:
        shutil.copy(os.path.join(TRAIN_IMAGES_DIR, img), os.path.join(selected_train_images_dir, img))
        label_file = os.path.splitext(img)[0] + '.txt'
        src_label_path = os.path.join(TRAIN_LABELS_DIR, label_file)
        dst_label_path = os.path.join(selected_train_labels_dir, label_file)
        if os.path.exists(src_label_path):
            shutil.copy(src_label_path, dst_label_path)

    # Update YAML file
    train_images_path = os.path.abspath(selected_train_images_dir)
    val_images_path = os.path.abspath(SELECTED_TEST_IMAGES_DIR)

    # Update the YAML file path for this iteration
    yaml_dir = 'week-06-portfolio/yaml'
    if not os.path.exists(yaml_dir):
        os.makedirs(yaml_dir)
        
    yaml_file_path_iter = f'week-06-portfolio/yaml/graffiti_iter_{iteration}.yaml'
    create_yaml_file(yaml_file_path_iter, train_images_path, val_images_path, nc, class_names)

    # Load the model from previous iteration
    if iteration == 1:
        model = YOLO('week-06-portfolio/models/yolov5su.pt')  # Start with pre-trained YOLOv5su model
    else:
        previous_model_path = f'week-06-portfolio/runs/train/graffiti_detection_iter_{iteration - 1}/weights/best.pt'
        model = YOLO(previous_model_path)

    # Train the model with 5 epochs
    model.train(
        data=yaml_file_path_iter,
        epochs=5,
        imgsz=640,
        batch=16,
        project='week-06-portfolio/runs/train',
        name=f'graffiti_detection_iter_{iteration}',
        device=device
    )

    # Evaluate the model
    output_images_dir_iter = f'week-06-portfolio/evaluation_images_iter_{iteration}'
    if not os.path.exists(output_images_dir_iter):
        os.makedirs(output_images_dir_iter)

    df_results = evaluate_model(model, SELECTED_TEST_IMAGES_DIR, SELECTED_TEST_LABELS_DIR, output_images_dir_iter)
    df_results.to_csv(f'week-06-portfolio/evaluation_results_iter_{iteration}.csv', index=False)

    # Check IoU threshold
    over_threshold = df_results[df_results['IoU_value'] > iou_threshold]
    if len(over_threshold) / len(df_results) >= 0.8:
        print(f"IoU threshold met in iteration {iteration}")
        model.save(f'week-06-portfolio/models/yolov5s_graffiti_iter_{iteration}.pt')
        satisfied = True
    else:
        print(f"IoU threshold not met in iteration {iteration}")

    # Prepare for next iteration
    iteration += 1


if not satisfied:
    print("IoU threshold not met after all iterations.")
else:
    model.save('week-06-portfolio/models/yolov5s_graffiti_final.pt')
    print("Training completed successfully.")




Starting iteration 1
New https://pypi.org/project/ultralytics/8.2.93 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=week-06-portfolio/models/yolov5su.pt, data=week-06-portfolio/yaml/graffiti_iter_1.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=mps, workers=8, project=week-06-portfolio/runs/train, name=graffiti_detection_iter_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None,

train: Scanning /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/week-06-portfolio/labels/train_selected_iter_1... 761 images, 0 backgrounds, 0 corrupt: 100%|██████████| 761/761 [00:00<00:00, 3410.07it/s]

train: New cache created: /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/week-06-portfolio/labels/train_selected_iter_1.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/labels/test_selected.cache... 40 images, 0 backgrounds, 0 corrupt: 100%|██████████| 40/40 [00:00<?, ?it/s]

Plotting labels to week-06-portfolio/runs/train/graffiti_detection_iter_1/labels.jpg... 


...


val: Scanning /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/datasets/11/labels/test_selected.cache... 40 images, 0 backgrounds, 0 corrupt: 100%|██████████| 40/40 [00:00<?, ?it/s]

Plotting labels to week-06-portfolio/runs/train/graffiti_detection_iter_31/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 69 weight(decay=0.0), 76 weight(decay=0.0005), 75 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to week-06-portfolio/runs/train/graffiti_detection_iter_31
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/5         0G     0.6179     0.6647     0.9591         69        640:  12%|█▏        | 3/25 [00:08<01:04,  2.94s/it]


RuntimeError: MPS backend out of memory (MPS allocated: 6.87 GB, other allocations: 11.24 GB, max allowed: 18.13 GB). Tried to allocate 24.61 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [4]:
def move_items_and_folders(source_dir, target_dir, exclude_dirs):
    """
    Moves all files and folders from source_dir to target_dir, excluding any folders listed in exclude_dirs.

    :param source_dir: Path to the source directory.
    :param target_dir: Path to the target directory where items will be moved.
    :param exclude_dirs: List of directory names to exclude from moving.
    """
    # Ensure the source directory exists
    if not os.path.exists(source_dir):
        print(f"Error: Source directory does not exist: {source_dir}")
        return

    # Create the target directory if it does not exist
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
        print(f"Created target directory: {target_dir}")
    else:
        print(f"Target directory already exists: {target_dir}")

    # Iterate over all items in the source directory
    for item in os.listdir(source_dir):
        item_path = os.path.join(source_dir, item)

        # Skip the target directory itself to prevent recursive moving
        if os.path.abspath(item_path) == os.path.abspath(target_dir):
            print(f"Skipped target directory itself: '{item}'")
            continue

        # Check if the item is a directory
        if os.path.isdir(item_path):
            if item in exclude_dirs:
                print(f"Excluded directory: '{item}'.")
                continue  # Skip excluded directories

            # Define the destination path
            dest_path = os.path.join(target_dir, item)

            # Check if destination already exists
            if os.path.exists(dest_path):
                print(f"Skipped: Destination already exists for folder '{item}'.")
                continue

            try:
                # Move the directory
                shutil.move(item_path, dest_path)
                print(f"Moved folder '{item}' to '{target_dir}'.")
            except Exception as e:
                print(f"Error moving folder '{item}': {e}")

        elif os.path.isfile(item_path):
            # If the item is a file, move it to the target directory
            dest_path = os.path.join(target_dir, item)

            # Check if destination file already exists
            if os.path.exists(dest_path):
                print(f"Skipped: Destination already exists for file '{item}'.")
                continue

            try:
                shutil.move(item_path, dest_path)
                print(f"Moved file '{item}' to '{target_dir}'.")
            except Exception as e:
                print(f"Error moving file '{item}': {e}")

        else:
            # For any other types of items (e.g., symbolic links), skip them
            print(f"Skipped non-directory, non-file item: '{item}'.")

In [5]:
WEEK_6_DIR = f'{HOME_DIR}/week-06-portfolio'
    
# Define the target 'train' directory
TARGET_TRAIN_DIR = os.path.join(WEEK_6_DIR, 'train')

# Define directories to exclude
EXCLUDE_DIRS = ['code', 'docs', 'requirements']

# Call the function to move folders
move_items_and_folders(WEEK_6_DIR, TARGET_TRAIN_DIR, EXCLUDE_DIRS)

Created target directory: /Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/train
Moved file 'evaluation_results_iter_2.csv' to '/Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/train'.
Moved folder 'evaluation_images' to '/Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/train'.
Moved file 'evaluation_results_iter_3.csv' to '/Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/train'.
Moved file 'evaluation_results_iter_1.csv' to '/Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/train'.
Moved folder 'evaluation_images_iter_6' to '/Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/train'.
Moved folder 'evaluation_images_iter_1' to '/Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/train'.
Moved file 'evaluation_results_iter_4.csv' to '/Users/anhdang/Documents/Github/COS40007

# Test Run on URL

In [13]:
model = YOLO('/Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/week-06-portfolio/train/runs/train/graffiti_detection_iter_30/weights/best.pt')

In [ ]:
source = 'https://videos.pexels.com/video-files/4543511/4543511-hd_1080_1920_25fps.mp4'
results = model.track(source, save=True, project=f'{HOME_DIR}/week-06-portfolio/results', tracker="bytetrack.yaml")

In [54]:
def extract_video_id(url):
    pattern = r'-([\d]+)/$'
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    else:
        return None
    
def get_video_name(video_url):
    pattern = r'/([\d]+-[\w]+_[\d]+_[\d]+_[\d]+fps\.mp4)$'
    match = re.search(pattern, video_url)
    if match:
        return match.group(1)
    else:
        return None


In [56]:
# TUTOR MAY WANT TO REPLACE WITH YOUR OWN API KEY!
PEXELS_API = ''

# Function to get the HD video link
def get_hd_video_link(video_id):
    url = f"https://api.pexels.com/videos/videos/{video_id}"
    command = f'curl -H "Authorization: {PEXELS_API}" {url}'
    response = subprocess.run(command, shell=True, capture_output=True, text=True)
    
    try:
        data = json.loads(response.stdout)
        # Look for the hd link in video_files
        for video_file in data['video_files']:
            if video_file['quality'] == 'hd':
                return video_file['link']
    except json.JSONDecodeError as e:
        print(f"Failed to retrieve video data for ID: {video_id}")
        return None

In [57]:
# List of URLs
urls = [
    "https://www.pexels.com/video/a-door-with-graffiti-on-it-is-shown-4543511/",
    "https://www.pexels.com/video/busy-street-footage-854181/",
    "https://www.pexels.com/video/graffiti-painted-on-the-train-station-wall-3413463/",
    "https://www.pexels.com/video/a-man-writing-on-a-wall-with-a-marker-9724130/"
]

# Extract and print HD video links for each URL
for url in urls:
    video_id = extract_video_id(url)
    if video_id:
        hd_link = get_hd_video_link(video_id)
        if hd_link:
            print(f"{hd_link}")
            print(f'{get_video_name(hd_link)}')
    else:
        print(f"Could not extract video ID from URL: {url}")


https://videos.pexels.com/video-files/4543511/4543511-hd_720_1280_50fps.mp4
4543511-hd_720_1280_50fps.mp4
https://videos.pexels.com/video-files/854181/854181-hd_1920_1080_25fps.mp4
854181-hd_1920_1080_25fps.mp4
https://videos.pexels.com/video-files/3413463/3413463-hd_1920_1080_30fps.mp4
3413463-hd_1920_1080_30fps.mp4
https://videos.pexels.com/video-files/9724130/9724130-hd_1440_1080_30fps.mp4
9724130-hd_1440_1080_30fps.mp4


In [ ]:
model = YOLO('/Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering/week-06-portfolio/train/runs/train/graffiti_detection_iter_30/weights/best.pt')
for url in urls:
    video_id = extract_video_id(url)
    if video_id:
        hd_link = get_hd_video_link(video_id)
        if hd_link:
            print(f"Proceessing: {hd_link}")
            model.track(hd_link, save=True, project=f'{HOME_DIR}/week-06-portfolio/results', conf=0.5, iou=0.9,  tracker="bytetrack.yaml")
            video_name = get_video_name(hd_link)
            if os.path.exists(f'{HOME_DIR}/{video_name}'):
                os.remove(f'{HOME_DIR}/{video_name}')
    else:
        print(f"Could not extract video ID from URL: {url}")
        